<a href="https://colab.research.google.com/github/jmelendezgeo/pagina12_scraper/blob/main/pagina12_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# Importamos las librerias necesarias
import requests
from bs4 import BeautifulSoup
from IPython.display import Image
import pandas as pd

In [62]:
def main(url):
    links_secciones = obtener_secciones(url)
    notas = obtener_notas(links_secciones)
    data = _obtener_data(notas)
    _save_data(data)


def obtener_secciones(url):
  '''
  Funcion que recibe la url y retorna una lista con los enlaces a las secciones
  '''

  # Manejo de errores al tener problemas con la url
  try:
    # Solicitud a url recibida
    request = requests.get(url)

    # Si status code es exitoso procedemos
    if request.status_code != 200:
      print('status code no exitoso')     

    else:
      # Creamos sopa con informacion de la pagina
      soup=BeautifulSoup(request.text,'lxml')

      # Tomamos las secciones y guardamos
      secciones = soup.find('ul',attrs={'horizontal-list main-sections hide-on-dropdown'}).find_all('li')

      # Guardamos los links de cada seccion en una lista
      links_secciones = [seccion.a.get('href') for seccion in secciones]

      # Retornamos la lista de secciones. Cada elemento de la lista es un enlace a una seccion
      return links_secciones

  except Exception as e:
    print('No se pudo obtener seccion. Error: ')
    print(e)
    print('\n')


def obtener_notas(soup):
  '''
  Funcion que recibe un objeto de BeautifulSoup de una página de una sección y 
  devuelve una lista de URLs a las notas de esa sección
  '''
  lista_notas=[]

  # Obtener el listado de articulos
  article_list = soup.find_all('div',attrs={'class':'article-item__content'})
  for article in article_list:
    lista_notas.append(article.a.get('href'))
  return lista_notas


def obtener_enlaces(links):
  '''
  Funcion que recibe una lista donde cada elemento es un enlace. Obtiene las noticias
  de cada enlace y retorna una lista con los enlaces de las notas en la primera página de cada seccion
  '''
  notas = []
  for link in links:
    try:
      request = requests.get(link)
      if request.status_code == 200:
        soup = BeautifulSoup(request.text,'lxml')
        notas.extend(obtener_notas(soup))
      else:
        print('No se pudo obtener la seccion')
    except:
      print('No se pudo obtener la seccion')
    
  return notas


def obtener_data(s_nota):
  '''
  Funcion que recibe el  enlace de la nota y larecorre para guardar la informacion 
  y retornarla en un diccionario
  '''
  ret_dict = {}

  # Extraemos el titulo
  titulo = s_nota.find('h1',attrs={'class':'article-title'})
  if titulo:
    ret_dict['titulo'] = titulo.text
  else:
    ret_dict['titulo'] = None

  # Extraemos la fecha del articulo
  fecha = s_nota.find('span', attrs={'pubdate':'pubdate'})
  if fecha:
    ret_dict['fecha'] = fecha.get('datetime')
  else:
    ret_dict['fecha'] = None

  # Extraemos la volanta
  volanta = s_nota.find('h2',attrs={'class':'article-prefix'})
  if volanta:
    ret_dict['volanta'] = volanta.text
  else:
    ret_dict['volanta'] = None
        
  # Extraemos el autor  
  autor = s_nota.find('div',attrs={'class':'article-author'})
  if autor:
    ret_dict['autor'] = autor.text
  else:
    ret_dict['autor'] = None

  # Extraemos resumen
  resumen = s_nota.find('div',attrs={'class':'article-summary'})
  if resumen: 
    ret_dict['resumen'] = resumen.text
  else:
    ret_dict['resumen'] = None

  # Extraemos imagen
  media = s_nota.find('div',attrs={'class':'article-main-media-image'})
  if media:
    imagenes = media.find_all('img')
    if len(imagenes) == 0:
      print('No se encontraron imagenes')
    else:
      imagen = imagenes[-1]
      img_src = imagen.get('data-src')
      try:
        img_request = requests.get(img_src)
        if img_request:
          ret_dict['imagen'] = img_request.content
        else:
          ret_dict['imagen'] = None
      except:
        print('No se encontro imagen.')
  else:
    print('No se encontro media')

  # Extraemos el cuerpo
  cuerpo = s_nota.find('div',attrs={'class':'article-text'})
  if cuerpo:
    ret_dict['texto'] = cuerpo.get_text()
  else:
    ret_dict['texto'] = None
  
  return ret_dict


In [63]:
def scrape_nota(url):
  try:
    nota = requests.get(url)
  except Exception as e:
    print('Error scrapeando url ', url)
    print(e)
    return None
  
  if nota.status_code !=200:
    print(f'Error obteniendo nota {url} ')
    print(f'Status code: {nota.status_code}')
    return None

  s_nota = BeautifulSoup(nota.text,'lxml')
  ret_dict = obtener_data(s_nota)
  ret_dict['url'] = url


  return ret_dict

In [64]:
if __name__ == '__main__':
  url = 'https://www.pagina12.com.ar'

  lista_secciones= obtener_secciones(url)
  notas = obtener_enlaces(lista_secciones)

  data = []
  for i, nota in enumerate(notas):
    print(f'Scrapeando nota {i+1}/{len(notas)}')
    data.append(scrape_nota(nota))





Scrapeando nota 0/66
Scrapeando nota 1/66
Scrapeando nota 2/66
Scrapeando nota 3/66
Scrapeando nota 4/66
Scrapeando nota 5/66
Scrapeando nota 6/66
Scrapeando nota 7/66
Scrapeando nota 8/66
Scrapeando nota 9/66
Scrapeando nota 10/66
Scrapeando nota 11/66
Scrapeando nota 12/66
Scrapeando nota 13/66
Scrapeando nota 14/66
Scrapeando nota 15/66
Scrapeando nota 16/66
Scrapeando nota 17/66
Scrapeando nota 18/66
Scrapeando nota 19/66
Scrapeando nota 20/66
Scrapeando nota 21/66
Scrapeando nota 22/66
Scrapeando nota 23/66
Scrapeando nota 24/66
Scrapeando nota 25/66
Scrapeando nota 26/66
Scrapeando nota 27/66
Scrapeando nota 28/66
Scrapeando nota 29/66
Scrapeando nota 30/66
Scrapeando nota 31/66
Scrapeando nota 32/66
Scrapeando nota 33/66
Scrapeando nota 34/66
Scrapeando nota 35/66
Scrapeando nota 36/66
Scrapeando nota 37/66
Scrapeando nota 38/66
Scrapeando nota 39/66
Scrapeando nota 40/66
Scrapeando nota 41/66
Scrapeando nota 42/66
Scrapeando nota 43/66
Scrapeando nota 44/66
Scrapeando nota 45/6

# Terminamos. Aqui tienes tus noticias:

In [66]:
pd.DataFrame(data)

,titulo,fecha,volanta,autor,resumen,imagen,texto,url
0,Santiago Cafiero y Eduardo De Pedro recibieron...,2021-03-25,Para avanzar en un plan conjunto de trabajo y ...,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"El jefe de Gabinete, Santiago Cafiero, recibió...",https://www.pagina12.com.ar/331781-santiago-ca...
1,La irónica respuesta de José Luis Gioja a Maur...,2021-03-25,¿Juntos por el Cambio es peronista?,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,El expresidente del Partido Justicialista (PJ)...,https://www.pagina12.com.ar/331778-la-ironica-...
2,Cristina Kirchner elogió la mirada de Horacio ...,2021-03-25,Contrastó su actitud con la de Macri,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,La vicepresidenta Cristina Kirchner escribió e...,https://www.pagina12.com.ar/331745-cristina-ki...
3,También fue eclesiástico,2021-03-25,45 años del Golpe,Por Antonio “Tony” Fenoy,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"Hacer memoria a 45 años del golpe cívico, ecle...",https://www.pagina12.com.ar/331743-tambien-fue...
4,Jujuy adelantó sus elecciones,2021-03-25,Se realizarán el 27 de junio,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,El Gobierno de Jujuy se sumó a la decisión que...,https://www.pagina12.com.ar/331737-jujuy-adela...
...,...,...,...,...,...,...,...,...
61,El porqué de los tatuajes,2021-03-11,Entrevista a la psicoanalista Hilda Catz,Por Oscar Ranzani,La moda no es el único disparador del deseo y ...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"Si bien parecen una moda, los tatuajes tienen ...",https://www.pagina12.com.ar/328718-el-porque-d...
62,8M: cómo transformar el trauma en celebración ...,2021-03-11,A propósito del Día de la Mujer,Por Graciela Trejo y Cristian Rodríguez,La movilización en Bariloche dispara una refle...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,La convocatoria por el Día Internacional de la...,https://www.pagina12.com.ar/328719-8-m-como-tr...
63,Mujeres y pandemia,2021-03-04,La concientización de lo femenino,Por Gladis Tripcevich Piovano,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Cuando se escriba la historia del año 2020 y l...,https://www.pagina12.com.ar/327347-mujeres-y-p...
64,Ultraje y violencia: libertad y expresión,2021-03-04,Acerca de las bolsas mortuorias en el acto opo...,Por Julián Ferreyra,A partir de la comparación de las imágenes de ...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,“¡Cuantos progresos hemos hecho! En la Edad Me...,https://www.pagina12.com.ar/327364-ultraje-y-v...
